Quantum mechanics, since its inception, has remained a cornerstone of modern physics. Among
its many intricate principles, quantum entanglement stands out as a quintessential phenomenon,
underlying a myriad of both foundational discussions and practical applications. The capability to
determine if a quantum state is entangled is not merely of theoretical interest, but is pivotal for
advancing quantum technologies. As a result, the field of quantifying and detecting entanglement
has been a very active. One known function called ’Entanglement Collectibility’ has been proposed
[1, 2], as an essential metric to ascertain the entanglement of quantum states.

The rigorous mathematical landscape of quantum mechanics has necessitated the adoption of sophis-
ticated computational techniques. Within this milieu, the Gradient Descent algorithm, originating
from the domain of optimization in machine learning, presents a promising approach. Gradient
Descent’s ability to navigate complex functional spaces by iteratively adjusting parameters makes
it apt for estimating intricate quantum functions.

In this research endeavor, we systematically employ various variants of the Gradient Descent
algorithm to estimate Entanglement Collectibility. Beyond mere estimation, this work critically
evaluates the performance of these variants within the challenging context of quantum systems.
Moreover, the thesis extends the application of these algorithms to derive significant quantities
with experimental relevance, particularly sets of mutually orthogonal separable set of states and the
associated observables. Through methodical computational examinations and robust analyses, this
work aims to fortify the bridge between classical computational strategies and quantum mechanics,
thereby contributing a refined set of tools to the quantum research paradigm.


In [1]:
# Library imports
# %matplotlib notebook

import random
import cmath,math
from pprint import pprint
import copy
import time
import itertools
from itertools import count
import sys
from tabulate import tabulate
from itertools import chain
from itertools import combinations_with_replacement

#numpy,pandas and matplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.animation as animation
from IPython.display import display, clear_output
from mpl_toolkits.mplot3d import Axes3D

#Qutip imports
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, 
                  destroy,expect, fock, fock_dm, mesolve, qeye, sigmax, 
                  sigmay,sigmaz, tensor, thermal_dm,dimensions,
                  rand_dm,rand_unitary,partial_transpose,
                  hadamard_transform,projection,phase_basis)

import qutip

from functools import reduce

In [ ]:
'''
Helper functions for generating random unitaries, calculating collectibility and gradient. 
To generate random parameters with the prescribed range. 
'''
def generate_Omega(delta):
    Omega = random.uniform(0,delta)
    return Omega

def generate_alpha(delta):
    alpha = random.uniform(0,2*math.pi*delta)
    return alpha

def generate_phi(r,s,delta,Omega):
    phi = math.asin(Omega**(1/(2*r)))
    return phi

def generate_psi(r,s,delta):
    psi = random.uniform(0,2*math.pi*delta)
    return psi

def generate_chi(r,s,delta):
    chi = random.uniform(0,2*math.pi*delta)
    return chi

def para_i(r,s):
    return (s**2-2*r-1, s**2-2*r, (s+1)**2-2*(s+1)) 

In [ ]:
'''
    These functions deal with matrix multiplications and tensor multiplication of a list of matrices. tge tensor_product_right and tensor_product_left function give a list of tensor products, each i-th element of which is the tensor product of first i elements of the inout matrix list. These are used to speed up the computation by storing the products before hand.
'''
def tensor_product(matrices):
    if len(matrices) < 2:
        raise ValueError("At least two matrices are required.")

    return reduce(np.kron, matrices)

def tensor_product_left(tensors):
    if len(tensors) < 2:
        raise ValueError("At least two tensors are required.")

    result = [tensors[0]]
    for i in range(1, len(tensors)):
        result.append(np.kron(result[i-1], tensors[i]))

    return result

def tensor_product_right(tensors):
    if len(tensors) < 2:
        raise ValueError("At least two tensors are required.")

    result = [tensors[-1]]
    for i in range(len(tensors)-2, -1, -1):
        result.insert(0, np.kron(tensors[i], result[0]))

    return result

def multiply_right_matrices(matrices):
    if len(matrices) < 2:
        raise ValueError("At least two matrices are required.")

    result = [matrices[-1]]
    for i in range(len(matrices)-2, -1, -1):
        result.insert(0, np.matmul(matrices[i], result[0]))

    return result

def Series_matrix_multiply(matrix_lst):
    return reduce(lambda a,b:np.matmul(a,b),matrix_lst)
\end{lstlisting}

\subsubsection{\underline{Activation function}}
\label{code:Activation function}
\begin{lstlisting}[language = Python]

In [ ]:
'''
These functions deploy the activation function, to convert the original parameters to their sigmoid form and inverse as well.
'''
def min_max_para(symbol,delta):
    if symbol=='alpha':
        para_min = 0
        para_max = 2*math.pi*delta
    elif symbol == 'Omega':
        para_min = 0
        para_max = delta
    elif symbol == 'psi':
        para_min = 0
        para_max = 2*math.pi*delta
    elif symbol == 'chi':
        para_min = 0
        para_max = 2*math.pi*delta
    
    return para_min,para_max

def sigmoid_function(symbol,num,delta):
    try:
        para_min,para_max = min_max_para(symbol,delta)
        sigm_num = (1/(1+math.exp(-num)))*para_max
        return sigm_num
    except Exception as err:
        print(f'{type(err)}: Error: {err} \n sigmoid function error for symbol:{symbol}, num:{num}')

def invert_sigmoid_function(symbol,num,delta):
    para_min,para_max = min_max_para(symbol,delta)
    invert_sigm_num = math.log(num/(para_max-num))
    return invert_sigm_num  

In [ ]:
# CUE parameter generator function -
def CUE_parameter_generator(size,delta):
    '''
    input: 
    size = N (size of one side of the random unitary matrix 
    being produced), depending on which, this function return a parameter 
    array of size. 
    
    delta = another user defined variable for determining random unitary
    
    return:
    Returns an array of size (n-1)^2 with the structure -
    [alpha,
    
    phi(1,2),
    psi(1,2),
    chi(1,2),
    
    phi(2,3),
    psi(2,3),
    phi(1,3),
    psi(1,3),
    chi(1,3),
    .
    .
    .
    phi(1,n),
    psi(1,n),
    chi(1,n),
    ]
    '''
    
    parameters = []
    alpha = generate_alpha(delta)
    sigm_alpha = invert_sigmoid_function('alpha',alpha,delta)
    parameters.append(['alpha',0,1,alpha,sigm_alpha])
    
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            Omega = generate_Omega(delta)
            sigm_Omega = invert_sigmoid_function('Omega',Omega,delta)
            parameters.append(['Omega',r,s,Omega,sigm_Omega])
            
            psi = generate_psi(r,s,delta)
            sigm_psi = invert_sigmoid_function('psi',psi,delta)
            parameters.append(['psi',r,s,psi,sigm_psi])
            
            if r==1:
                chi = generate_chi(r,s,delta)
                sigm_chi = invert_sigmoid_function('chi',chi,delta)
                parameters.append(['chi',r,s,chi,sigm_chi])
    return parameters

In [ ]:
def Mutually_Orthogonal_state_generator(system):
    '''
    Used to generate mutually orthogonal basis for a given system.
    Initially, the assumption is that all Hilbert spaces of the subsystem are of 
    the same dimension.
    '''
    N = min(system)
    MO_Basis_set = []
    for subsys_dim in system:
        MO_Basis_subset = [basis(subsys_dim, n) for n in range(subsys_dim)]
        MO_Basis_set.append(MO_Basis_subset)
    return MO_Basis_set

def Seperable_state_set_generator(MO_Basis_set):
    '''
    Used to produce N - Seperable state set.  
    Initially, the assumption is that all Hilbert spaces of the subsystem are of 
    the same dimension.
    '''
    Seperable_basis_set = []
    N = max([len(subsys) for subsys in MO_Basis_set])
    for n in range(N): 
        Seperable_basis_set.append(tensor([subsys[n] for subsys in MO_Basis_set]))
    return Seperable_basis_set

In [ ]:
def Elementary_Unitary_Matrices_Generator(size,r,s,delta,Omega, psi, chi):
    # Creating the Elementary 2D subsystem unitary matrix
    E = np.eye(size, dtype=complex)
    r-=1 
    s-=1
    
    phi = math.asin(Omega**(1/(2*(r+1))))
    
    E[r,r] = math.cos(phi) * cmath.exp(complex(0,psi))  
    E[r,s] = math.sin(phi) * cmath.exp(complex(0,chi))
    E[s,r] = -math.sin(phi) * cmath.exp(complex(0,-chi))
    E[s,s] = math.cos(phi) * cmath.exp(complex(0,-psi))
    
    return E

In [ ]:
'''
CUE_generator - 
First step is to work with unitary matrices. 
It will have a function to calculate random unitary matrix using Zyczkowski's method.
'''
def CUE_generator(size, parameters, delta):
    '''
    input: 
    
    1) size = it is the dimensionality (of one side) of the aimed 
    random unitary matrix.
    
    2) parameters = This is a list of parameters used to parametrize the 
    unitary matrix, necessary to generate CUE(Circular Unitary Ensembles)
    deterministically. It is crucial to keep track of these parameters to
    eventually work with gradient descent.
    
    return:
    1) return a unitary matrix of given size, with given parameters and 
    delta.
    '''

    
    alpha = float(parameters[0][3])
    
    Left = []
    Elementary_matrices = [cmath.exp(complex(0,alpha))]
    right = []
    
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            indexes = para_i(r,s)
            Omega = float(parameters[indexes[0]][3])
            psi = float(parameters[indexes[1]][3])
            if r==1:
                chi = float(parameters[indexes[2]][3])
            else:
                chi = 0
            E = Elementary_Unitary_Matrices_Generator(size,r,s,delta,Omega,psi,chi)

            if not Qobj(E).check_isunitary():
                raise Exception('U is not unitary.')

            Elementary_matrices.append(E)
    
    
    Left_multiply = Series_multiply(Elementary_matrices,'left')
    Right_multiply = Series_multiply(Elementary_matrices,'right')
    U = Series_multiply(Elementary_matrices)
    
    if Qobj(U).check_isunitary():
        return (U,Elementary_matrices, Left_multiply, Right_multiply)
    else:
        raise Exception('U is not unitary.')

In [ ]:
# Calculating Y values for given parameters.
def Y_Calculator(sep_state_set,U,rho):
    '''
    Inputs: 
    sep_state_set = list of seperable state vectors
    U_lst = the unitary to be applied for optimization
    rho = the state of the system being tested for entanglement, given as a density matrix
    
    Outputs:
    (Y,Y_list,left_Y, right_Y,Yij_matrix)
    
    where:
    Y = The final calcuated value of Y, given value inputs.
    
    Y_list = The list of 'Y multiplier' 
    
    left_Y = The list of multiplied <Y(i,j)> from the left: 
    [(<Y(1,1)>), (<Y(1,1)><Y(1,2)>),....,(<Y(1,1)>...<Y(N,N)>)]
    
    right_Y = The list of multiplied <Y(i,j)> from the right:
    [(<Y(N,N)>), (<Y(N,N)><Y(N-1,N)>),....,(<Y(N,N)>...<Y(1,1)>)]
    
    '''
    U = Qobj(U)
    U_dag = U.dag()
    
    rho = Qobj(rho)

    system_dim = len(sep_state_set)
   

    Y_list = []
    Yij_list = []
    Yij_matrix = []
    
    for i in range(system_dim):
        Yij_matrix.append([])
        for j in range(system_dim):
            Y_multiplier = reduce(np.matmul,[np.array(sep_state_set[i].dag().full()),np.array(U_dag.full()),np.array(rho.full()),np.array(U.full()),np.array(sep_state_set[j].full())])
            Yij_list.append([Y_multiplier[0][0],i,j])
            Yij_matrix[i].append(Y_multiplier[0][0])
            Y_list.append(Y_multiplier[0][0])
    
    Y = reduce(lambda a,b:a*b, Y_list)
    Y = Y**(1/system_dim)

    left_Y = Series_multiply(Y_list,'left')
    right_Y = Series_multiply(Y_list,'right')
    
    return (Y,Yij_list,left_Y,right_Y,Yij_matrix)

In [ ]:
def calculate_dEdp(E,para,para_i,delta):
'''
For calculation of the dE/dp, derivative of Elementary 2D unitary matrix with respect to the parameter being considered.
'''
    symbol = para[para_i][0]
    r = int(para[para_i][1])
    s = int(para[para_i][2])
    parameter_value = float(para[para_i][3])
    sigm_parameter_value = float(para[para_i][4])
    
    para_min,para_max = min_max_para(symbol,delta)
    
    try:
        exp_para = math.exp(-sigm_parameter_value)
        dsigmapara_dpara = (exp_para*para_max)/((1+exp_para)**2)
    except Exception as err:
        print(f'{type(err)}: Error: {err}\n exp_para:{exp_para},\n para_max:{para_max},\n sigm_parameter_value:{sigm_parameter_value}')
        
    chi=0
    dEdp = np.zeros(E.shape, dtype=complex)
    r_pow = r
    r-=1 
    s-=1
        
    try: 
        if symbol == 'Omega':
            Omega = parameter_value
            psi = float(para[para_i+1][3])
            if r==1:
                chi = float(para[para_i+2][3])

            power = 1/(2*(r_pow))
            z = Omega**(power)  

            dEdp[r,r] = cmath.exp(complex(0,psi)) * -math.sin(math.asin(z)) * 1/(math.sqrt(1-z**2)) * power * (Omega**(power-1)) * dsigmapara_dpara
            dEdp[r,s] = cmath.exp(complex(0,chi)) * math.cos(math.asin(z)) * 1/(math.sqrt(1-z**2)) * power * (Omega**(power-1)) * dsigmapara_dpara
            dEdp[s,r] = -cmath.exp(complex(0,-chi)) * math.cos(math.asin(z)) * 1/(math.sqrt(1-z**2)) * power * (Omega**(power-1)) * dsigmapara_dpara
            dEdp[s,s] = cmath.exp(complex(0,-psi)) * -math.sin(math.asin(z)) * 1/(math.sqrt(1-z**2)) * power * (Omega**(power-1)) * dsigmapara_dpara

        elif symbol == 'psi':
            Omega = float(para[para_i-1][3])
            psi = parameter_value
            if r==1:
                chi = float(para[para_i+1][3])

            power = 1/(2*(r_pow))
            z = Omega**(power)
            phi = math.asin(z)        

            dEdp[r,r] = complex(0,1) * math.cos(phi) * cmath.exp(complex(0,psi)) *  dsigmapara_dpara
            dEdp[r,s] = 0
            dEdp[s,r] = 0
            dEdp[s,s] = -complex(0,1) * math.cos(phi) * cmath.exp(complex(0,-psi)) *  dsigmapara_dpara

        elif symbol == 'chi':
            Omega = float(para[para_i-2][3])
            psi = float(para[para_i-1][3])
            if r==1:
                chi = parameter_value

            power = 1/(2*(r_pow))
            z = Omega**(power)
            phi = math.asin(z)

            dEdp[r,r] = 0  
            dEdp[r,s] = complex(0,1) * math.sin(phi) * cmath.exp(complex(0,chi)) *  dsigmapara_dpara
            dEdp[s,r] = -complex(0,1) * -math.sin(phi) * cmath.exp(complex(0,-chi)) *  dsigmapara_dpara
            dEdp[s,s] = 0
    except Exception as err:
        print(f'{type(err)}: Error:{err}\n z: {z}')
        
    return dEdp

In [ ]:
def calculate_du_system_dp(u_system, para,para_i,sys_i,delta):
'''
For calculation of the du/dp, derivative of local unitary matrix with respect to the parameter p.
'''

    symbol = para[para_i][0]
    r = float(para[para_i][1])
    s = float(para[para_i][2])
    parameter_value = float(para[para_i][3])
    sigm_parameter_value = float(para[para_i][4])
    
    
    if symbol=='alpha':
        return complex(0,1)*sigm_parameter_value*u_system[0]
    
    elementary_index = int((s-1)*(s)/2 - (r-1))
    E = u_system[1][elementary_index]

    dEdp = calculate_dEdp(E,para,para_i,delta)
    
    if elementary_index == 0:
        du_system_dp = Series_multiply([dEdp,u_system[3][sys_i+1]])
    elif elementary_index == len(u_system[1])-1:
        du_system_dp = Series_multiply([u_system[2][elementary_index-1],dEdp])
    else:
        du_system_dp = Series_multiply([u_system[2][sys_i-1],dEdp,u_system[3][sys_i+1]])
        
    return du_system_dp

In [ ]:
def calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product,delta):
    '''
    For calculation of the dU/dp, derivative of global unitary (tensor product of all local unitaries) matrix with respect to the parameter p.
    '''
    u_system = U[sys_i]
    du_system_dp = calculate_du_system_dp(u_system, para,para_i,sys_i,delta)
    
    if sys_i == 0:
        dUdp = tensor_product([du_system_dp,right_tensor_product[sys_i+1]])
    elif sys_i == len(U)-1:
        dUdp = tensor_product([left_tensor_product[sys_i-1],du_system_dp])
    else:
        dUdp = tensor_product([left_tensor_product[sys_i-1],du_system_dp,right_tensor_product[sys_i+1]])
    
    return dUdp

In [ ]:
def calculate_Yij_dp_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product,delta):
    '''
    For calculation of the dYii/dp, derivative of the diagonal multipliers Yii (i=1,2,...,N), which constitute collectibility function Y, with respect to the parameter p.
    '''
    Total_U_dag = np.array(Total_U).transpose().conjugate()
    xi_a_dag = np.array(xi_a).transpose().conjugate()
      
    dUdp = calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product,delta)
    dUdp_dag = np.array(dUdp).transpose().conjugate()
    
    dYij_dp = Series_matrix_multiply([xi_a_dag,dUdp_dag,np.array(rho),Total_U,np.array(xi_b)])
    
    return dYij_dp[0][0]

def calculate_Yij_dp_off_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product,delta):
    '''
    For calculation of the dYij/dp, derivative of the non-diagonal multipliers Yij (i,j=1,2,...,N), which constitute collectibility function Y, with respect to the parameter p.
    '''

    Total_U_dag = np.array(Total_U).transpose().conjugate()
    xi_a_dag = np.array(xi_a).transpose().conjugate()
      
    dUdp = calculate_dUdp(U,para,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product,delta)
    dUdp_dag = np.array(dUdp).transpose().conjugate()
    
    term_a = Series_matrix_multiply([xi_a_dag,dUdp_dag,np.array(rho),Total_U,np.array(xi_b)])
    term_b = Series_matrix_multiply([xi_a_dag,Total_U_dag,np.array(rho),dUdp,np.array(xi_b)])
    
    dYij_dp = term_a + term_b
    
    return dYij_dp[0][0]

In [ ]:
def calculate_dY_dp_modified(sep_state_set,rho,U,para,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product,delta):
    '''
    For calculation of the dY/dp, derivative of the Collectibility function Y, with respect to the parameter p.
    '''
    y = Y[4]
    
    partial_diff_sum = 0
    abs = lambda f:math.sqrt(f.real**2+f.imag**2)
    
    for i in range(len(y)):
        for j in range(len(y[i])):
            xi_a = sep_state_set[i]
            xi_b = sep_state_set[j]

            if i==j:
                y_original = y[i][j]
                dYij_dp = calculate_Yij_dp_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product,delta)
                dYij_dp = 2*dYij_dp.real
                partial_diff_sum+=((dYij_dp)/abs(y_original))
            elif i<j:
                Yij = y[i][j]
                Yji = y[j][i]
                y_original = Yij*Yji
    
                dYij_dp = calculate_Yij_dp_off_diagonal(sep_state_set,rho,U,para,sys_i,para_i,xi_a,xi_b,Total_U,right_tensor_product,left_tensor_product,delta)
                dYij_dp = dYij_dp*Yji
                dYij_dp = 2*dYij_dp.real

                partial_diff_sum+=((dYij_dp)/((abs(y_original))**2))
            else:
                continue
    
    dYdp = abs(Y[0])*partial_diff_sum
    return dYdp.real

In [ ]:
def Gradient(sep_state_set,rho,U,parameters,delta):
    '''
    This function calculates Gradient of Collectibility function Y for each parameter in the parameter vector, returning a gradient vector.
    U is a list of local unitaries = [U_1,U_2,....,U_k]
    '''
    Local_unitary_list = [U[n][0] for n in range(len(U))]
    Total_U = tensor_product(Local_unitary_list)
    right_tensor_product = tensor_product_right(Local_unitary_list)
    left_tensor_product = tensor_product_left(Local_unitary_list)
    
    Y = Y_Calculator(sep_state_set, Total_U, rho)
    gradient = []
    for sys_i in range(len(parameters)):
        gradient.append([])
        for para_i in range(len(parameters[sys_i])):
            para = parameters[sys_i]
            dYdp = calculate_dY_dp_modified(sep_state_set,rho,U,para,Y,sys_i,para_i,Total_U,right_tensor_product,left_tensor_product,delta)
            gradient[sys_i].append(dYdp)
            
    return gradient

In [ ]:
'''
These are helper function for Gradient Descent fucntion:
'''
def shifting_sigm_parameters(parameters, shift_in_para, delta):
    for sys in range(len(parameters)):
        for para in range(len(parameters[sys])):
            a = parameters[sys][para][4]
            b = shift_in_para[sys][para]
            
            c = sigmoid_function(parameters[sys][para][0],a+b,delta)
            
            parameters[sys][para][3] = c
            parameters[sys][para][4] = a+b

    return parameters

def parameter_based_unitary(parameters,delta):
    U = []
    for p in parameters:
        size = int(math.sqrt(len(p)))
        
        uni = CUE_generator(size, p, delta)
        U.append(uni)
        
    return U


def Y_calculator_wrapper(sep_state_set, rho, U):
    
    Local_unitary_list = [U[n][0] for n in range(len(U))]
    
    Total_U = tensor_product(Local_unitary_list)
    
    Y = Y_Calculator(sep_state_set, Total_U,rho)
    return Y
    
'''
Important function! lising the hyperparameters for various gradient descent adaptations.
'''
def dynamic_stepsize():
    stepsize = 0.1
    
    momentum_decay_rate = 0.9
    RMSProp_decay_rate = 0.58
    
    Adam_Beta_momentum = 0.8
    Adam_Beta_RMS = 0.9
    
    return (stepsize,momentum_decay_rate,RMSProp_decay_rate,Adam_Beta_momentum,Adam_Beta_RMS)

In [ ]:
'''
This function runs the Gradient Descent of a given form (method) for given iterations and returns the list of collectibility estimates and corresponding paramters. Output of this function, after multiple runs, is used to give the final approximation of collectibility.
'''
def Gradient_Descent(system,rho,delta,max_iterations,return_parameters,method = 'RMSProp'):
    abs = lambda f:math.sqrt(f.real**2+f.imag**2)
    sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))
    
    parameters = []
    for n in system:
        size = n
        p = CUE_parameter_generator(size, delta)
        parameters.append(p)
    
    U = parameter_based_unitary(parameters,delta)
    
    Y = Y_calculator_wrapper(sep_state_set, rho, U)
    
    Y_old = abs(Y[0]).real
    Y_correction = [Y_old]
    
    Y_correction_max = max(Y_correction)
    
    if method in ['Random']:
        Y_correction = [Y_old,Y_old]
        if return_parameters:
            return Y_correction,(Y_correction_max,parameters)
        else:
            return Y_correction
    
    gradient_sum = np.zeros((len(parameters),len(parameters[0])))
    gradient_sum_square = np.zeros((len(parameters),len(parameters[0])))
    
    (stepsize,momentum_decay_rate,RMSProp_decay_rate,Adam_Beta_momentum,Adam_Beta_RMS) = dynamic_stepsize()
    nrm = math.inf
    stepsize_list = [0]
    
    if method in ['Gradient_Descent','AdaGrad','RMSProp']:
        while Y_correction_max == Y_correction[-1] and len(Y_correction)<=max_iterations:
            
            gradient = Gradient(sep_state_set,rho,U,parameters,delta) 
            gradient = np.array(gradient)
            nrm = np.linalg.norm(gradient)

            if method == 'Gradient_Descent':
                newstepsize = stepsize/nrm
                shift_in_paras = newstepsize*(gradient)

            elif method == 'AdaGrad':
                gradient_sum_square = gradient**2 + gradient_sum_square
                shift_in_paras = stepsize*(gradient/np.sqrt(gradient_sum_square))

            elif method == 'RMSProp':
                gradient_sum_square = (gradient**2)*(1-RMSProp_decay_rate) + gradient_sum_square*RMSProp_decay_rate
                shift_in_paras = stepsize*(gradient/np.sqrt(gradient_sum_square))
            
            new_parameters = shifting_sigm_parameters(parameters, shift_in_paras, delta)
            
            U = parameter_based_unitary(new_parameters,delta)
            Y_new = Y_calculator_wrapper(sep_state_set, rho, U)
            Y_new = abs(Y_new[0]).real

            Y_correction.append(Y_new)

            Y_correction_max = max(Y_correction_max,Y_new)

            parameters = new_parameters
        
    elif method in ['Momentum','Adam']:
        for _ in range(max_iterations):
            gradient = Gradient(sep_state_set,rho,U,parameters,delta) 
            gradient = np.array(gradient)

            if method == 'Momentum':
                gradient_sum = gradient + momentum_decay_rate*gradient_sum
                shift_in_paras = stepsize*gradient_sum

            elif method == 'Adam':
                gradient_sum = gradient*(1-Adam_Beta_momentum) + Adam_Beta_momentum*gradient_sum
                gradient_sum_square = (gradient**2)*(1-Adam_Beta_RMS) + gradient_sum_square*(Adam_Beta_RMS)
                shift_in_paras = stepsize*(gradient_sum/np.sqrt(gradient_sum_square))

            new_parameters = shifting_sigm_parameters(parameters, shift_in_paras, delta)

            U = parameter_based_unitary(new_parameters,delta)
            Y_new = Y_calculator_wrapper(sep_state_set, rho, U)
            Y_new = abs(Y_new[0]).real

            Y_correction.append(Y_new)

            Y_correction_max = max(Y_correction_max,Y_new)

            parameters = new_parameters
                
    if return_parameters:
        return Y_correction,(Y_correction_max,parameters)
    else:
        return Y_correction

In [ ]:
'''
These are helper function for the final test function. Rn is to calculate the R_n function. PPT_K2_limit and PPT_N2_limit are function to return PPT limits of collectibility for bipartite quNit  and multipartite qubit systems respectively.  
'''
def Rn(D,Tr,Tr2,N):
    E = Tr2/(Tr**2)
    D_ = D-N
   
    inside_sqrt = (D+D_-1)**2 + 4*D*D_*(E-1)

    P = (D+D_-1 - math.sqrt(inside_sqrt))/(2*D*D_)
    R = (((Tr)/(N))**N)*((1-D*P)**((N-1)/(2)))*((1-D_*P)**((N+1)/(2)))

    def PPT_K2_limit(N,K):
    return N**(-2*N)

def PPT_N2_limit(N,K):
    return 1/(16*(2**(K-1)-1))


In [ ]:
'''
This function is utilizing the bisection method to determin the critical purity for given collectibility limits.
'''
def bisection(f, a, b,N,K,Y, tolerance=1e-7, max_iterations=100):
    """
    Bisection method for finding the root of a function.
    
    :param f: Function whose root is to be found
    :param a: Lower bound of the interval
    :param b: Upper bound of the interval
    :param tolerance: Tolerance (default: 1e-7)
    :param max_iterations: Maximum number of iterations (default: 100)
    :return: The root of the function f
    """
    if f(a,N,K,Y) * f(b,N,K,Y) >= 0:
        raise ValueError('f(a) and f(b) must have opposite signs')
    
    for _ in range(max_iterations):
        c = (a + b) / 2
        if f(c,N,K,Y) == 0 or (b - a) / 2 < tolerance:
            return c
        if f(c,N,K,Y) * f(a,N,K,Y) < 0:
            b = c
        else:
            a = c
            
    raise Exception('Maximum iterations reached. No root found within tolerance.')

# Define the function for bisection
def Bisection_Rn(x,N,K,Y):
    D = N**K
    Tr=1
    return Rn(D,Tr,x,N) - Y


In [ ]:
def Tests_on_Collectibility(system, rho, IsWerner, delta, method_list, max_iterations=2**10,max_iterations_GD = 100, visual_interval=100,iteration_print_interval=10):
    '''
    This function uses all the functionality, utilizing Gradient descent using its various methods to determine Collectibility Estimate and gives a visualization of the the performance of the each methods and generates a report with all the results as promised in this thesis work.  
    Input: Give me a system to test, the density matrix (rho) of the state and Garadient Descent Varients from ['Gradient_Descent', 'Random', 'RMS Prop', 'AdaGrad', 'Adam', 'momentum' ] to test its collectibility.    
    Output: And then it detects NPPT states and characterizes the state in other ways possible using Entanglement collectibility.
    '''
    rho = np.array(rho)
    rho_sqr = np.matmul(rho,rho)
    
    limits_list = {}
    max_parameters = []
    Y_max = 0
    
    K = len(system)
    if K<=1:
        raise Exception("InputError: The system has no subsystems.")
        
    if all(list(i == system[0] for i in system)):
        N = system[0]
    else:
        raise Exception("InputError: The system has different dimensional subsystems.")
        
    D = N**K
    Trace = rho.trace().real
    if not math.isclose(Trace,1,rel_tol = 1e-4):
        raise Exception(f"InputError: Trace of Density Matrix is not 1. It is {Trace}. Invalid Density Matrix.")
        
    Purity = rho_sqr.trace().real
    General_Y_limit = Rn(D, Trace, Purity,N)
    
    limits_list['General_Y_Upper_Bound'] = General_Y_limit
    
    #Bisection variables: 
    a = (N**K)**(-1)
    b = 1
    tolerance = 1e-7
    max_bisection_iterations = 100
    
    if K == 2:
        PPT_limit = PPT_K2_limit(N,K)
        PPT_critical_Purity = bisection(Bisection_Rn, a, b,N,K,PPT_limit,tolerance,max_bisection_iterations)
        if IsWerner:
            werner_Collectibility = ymax_werner(N,alpha,amps)
            limits_list['Werner_Y'] = werner_Collectibility
            Message = f'The system is a werner state (K={K}), with each subsystem being N={N}-dimensional.'
            
    elif N==2:
        PPT_limit = PPT_N2_limit(N,K)
        PPT_critical_Purity = bisection(Bisection_Rn, a, b,N,K,PPT_limit,tolerance,max_bisection_iterations)
        Message = f'The system is a {K}-partite system, with each subsystem being N={N}-dimensional.'
    else:
        Message = f'The system is a {K}-partite system, with each subsystem being N={N}-dimensional.'
        
    limits_list['PPT_limit'] = PPT_limit
        
    if N==2 or K==2:
        if Purity<PPT_critical_Purity:
            Message = Message + f'\nPurity({round(Purity,4)}) is less than Critical Purity ({round(PPT_critical_Purity,4)}).\nHence, collectibility should not be able to detect NPPT. But we run the algorithm anyway, to be sure.'
        else:
            Message = Message + f'\nPurity({round(Purity,4)}) is more than Critical Purity ({round(PPT_critical_Purity,4)}). \nHence, we calculate collectibility to see if the state is NPPT.'
    print(Message)
        
    #Y_calculation
    methods_so_far = []
    max_Y = {}
    method_time = {}
    Collectibility_calculations = {}
    
    color_scheme = {'Werner_Y':'g','General_Y_Upper_Bound':'r','PPT_limit':'c'}
    animate = lambda k: l.set_data(t,max_Y[method])
    
    for method in method_list:
        methods_so_far.append(method)
        max_Y[method] = [0]
        
        method_iteration = 0
        method_time[method] = [0]

        print('Method - '+ method)
                
        for i in range(1, max_iterations + 1):
            if i%iteration_print_interval==0:
                print(i)
                
            t1 = time.time()
            GD,Y_max_parameters = Gradient_Descent(system,rho,delta,max_iterations_GD,True,method)

            t2 = time.time()
            
            method_time[method].append(method_time[method][-1]+t2-t1)
            
            max_Y[method].append(max(max_Y[method][-1],GD[-2]))
            
            if max_Y[method][-1] <= Y_max_parameters[0]:
                max_parameters = Y_max_parameters[1]
                Y_max = Y_max_parameters[0]
                
            # Visualizing
            if i%visual_interval==0:
                fig, ax = plt.subplots()
                
                t = method_time[method]
                for m in methods_so_far:
                    if max_Y[method][-1]<PPT_limit:
                        linclr = 'g'
                    else:
                        linclr = 'r'
                    l, = ax.plot(t,max_Y[m][:len(t)],linestyle = '-.',label=method)
    
                for key in limits_list:
                    plt.axhline(limits_list[key],color = color_scheme[key],label=key)

                plt.title(f'Iterations({i}/{max_iterations}) vs Y_collectibility ({method})')
                plt.ylabel('Y_collectibility')
                plt.xlabel(f'Time(s)')

                handles, _ = ax.get_legend_handles_labels()

                plt.legend(handles = handles, labels = methods_so_far, loc = 'best')
                
                if IsWerner:
                    method_accuracy = round(max_Y[method][-1]/werner_Collectibility*100, 2)
                    plt.annotate('Werner_Y' + f'({method_accuracy}%)', xy = (t[-1],werner_Collectibility)) 
                    
                for lmt in limits_list:
                    if lmt!='Werner_Y':
                        plt.annotate(lmt, xy = (t[-1],limits_list[lmt]))
                
                plt.annotate(method, xy = (i,round(max_Y[method][-1],4)))
                
                animate(len(t))
                clear_output(wait=True)
                print(Message+'\n')
                print('Method - '+method)
                display(fig)
                
        method_collectibility_max = max_Y[method][-1]
        Collectibility_calculations[method] = [method,method_collectibility_max,method_time[method][-1],len(method_time[method])-1]

    #Graph display
    clear_output(wait=True)
    print('\n'+Message+'\n')
    display(fig)

    #Report Display
    Final_Collectibility = [0,0,0,0]
    for method in methods_so_far:
        if Collectibility_calculations[method][1]>Final_Collectibility[1]:
            Final_Collectibility = Collectibility_calculations[method]
    
    Y_estimate_list = list(Collectibility_calculations.values())
    print(tabulate(Y_estimate_list, headers=["Method", "Y Estimates", "Time(s)", "Iterations"]))
            
    print(f'\nThe "{Final_Collectibility[0]}" method produced the\nMaximum Collectibility Estimate: {round(Final_Collectibility[1],6)}\nin {round(Final_Collectibility[2],2)} seconds \nwith {Final_Collectibility[3]} iterations.\n')
    
    print (tabulate(list(map(list, limits_list.items())), headers=["Limit", "Limit Value"]))
    
    if K==2 or N==2:
        if Final_Collectibility[1]>limits_list['PPT_limit']:
            print(f'\nFinal collectibility estimate is above the PPT threshold,\nhence NPPT state detected.')
        else:
            print(f'\nFinal collectibility estimate is below the PPT threshold,\nhence cannot be conclusive about the PPT condition for given state.')

    # Maximizing N Separable state set: 
    U_max = parameter_based_unitary(max_parameters,delta)
    Local_unitary_list = [U_max[n][0] for n in range(len(U_max))]
    Total_U_max = tensor_product(Local_unitary_list)
    sep_state_set = Seperable_state_set_generator(Mutually_Orthogonal_state_generator(system))
    
    max_pure_states = []
    for sep_state in sep_state_set:
        max_pure_states.append(Total_U_max*sep_state)
    
    mutuality_test_matrix = []
    for i in range(len(max_pure_states)):
        mutuality_test_matrix.append([])
        for j in range(len(max_pure_states)):
            dot_prod = Qobj(max_pure_states[i]).dag()*max_pure_states[j]
            if (cmath.isclose(dot_prod,0,abs_tol=1e-9)):
                mutuality_test_matrix[i].append(0)
            elif (cmath.isclose(dot_prod,1,abs_tol=1e-9)):
                mutuality_test_matrix[i].append(1)

    # Generating corresponding Observables:
    if np.allclose(mutuality_test_matrix,np.eye(N)):
        print('\nThe mutually Orthogonal Seperable state set maximizing the estimated Collectibility Y_max is: ')
        pprint(max_pure_states)
        
        print('\nThe corresponding observables are: ')
        
        Observables = []
        comb = combinations_with_replacement(list(range(len(max_pure_states))), 2)
        for pair in comb:
            (i,j) = pair
            if i==j:
                XiXj = Qobj(max_pure_states[j])*Qobj(max_pure_states[i]).dag()
                Observables.append(np.array(XiXj))
                print(i,j,XiXj)
            else:
                XiXj = Qobj(max_pure_states[j])*Qobj(max_pure_states[i]).dag()
                Obs = np.array((XiXj+XiXj.dag()).full())
                Obs_dag = complex(0,1)*np.array((XiXj-XiXj.dag()).full())
                Observables.append(Obs)
                Observables.append(Obs_dag)
                print(i,j,Obs)
                print(i,j,Obs_dag)
                
    else:
        raise Exception(f'\nThe maximizing Seperable state set is not mutually Orthogonal') 